# Running an Individual Test
This notebook shows how to run individual metrics or thresholds tests.

## Before you begin

To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready.

If you don't already have one, you should also [create a documentation project](https://docs.validmind.ai/guide/create-your-first-documentation-project.html) on the ValidMind platform. You will use this project to upload your documentation and test results.

## Install the client library

In [ ]:
%pip install --upgrade validmind

## Initialize the client library

In a browser, go to the **Client Integration** page of your documentation project and click **Copy to clipboard** next to the code snippet. This code snippet gives you the API key, API secret, and project identifier to link your notebook to your documentation project.

::: {.column-margin}
::: {.callout-tip}
This step requires a documentation project. [Learn how you can create one](https://docs.validmind.ai/guide/create-your-first-documentation-project.html).
:::
:::

Next, replace this placeholder with your own code snippet:

In [ ]:
## Replace with code snippet from your documentation project ##

import validmind as vm

vm.init(
  api_host = "https://api.prod.validmind.ai/api/v1/tracking",
  api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
  api_secret = "API_SECRET",
  project = "xxxxxxxxxxxxxxxxxxxxxxxxx"
)

In [ ]:
import pandas as pd
import xgboost as xgb

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline

## Load the Demo Dataset

In [ ]:
# You can also import customer_churn like this:
from validmind.datasets.classification import customer_churn as demo_dataset
# from validmind.datasets.classification import taiwan_credit as demo_dataset

df = demo_dataset.load_data()

### Prepocess the Raw Dataset

In [ ]:
train_df, validation_df, test_df = demo_dataset.preprocess(df)

## Train a Model for Testing

We train a simple customer churn model for our test.

In [ ]:
x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

model = xgb.XGBClassifier(early_stopping_rounds=10)
model.set_params(
    eval_metric=["error", "logloss", "auc"],
)
model.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False,
)

## Import and Run the Individual Test

### Initialize ValidMind objects

In [ ]:
vm_train_ds = vm.init_dataset(
    dataset=train_df,
    type="generic",
    target_column=demo_dataset.target_column
)

vm_test_ds = vm.init_dataset(
    dataset=test_df,
    type="generic",
    target_column=demo_dataset.target_column
)

vm_model = vm.init_model(
    model,
    train_ds=vm_train_ds,
    test_ds=vm_test_ds,
)


### Import the individual test

In [ ]:
from validmind.vm_models.test_context import TestContext
from validmind.tests.model_validation.sklearn.TrainingTestDegradation import TrainingTestDegradation


### Pass the required inputs and config parameters

In [ ]:
test_context = TestContext(model=vm_model)
ws_diagnostic = TrainingTestDegradation(test_context)

### Run the test

In [ ]:
ws_diagnostic.run()

### Display results

In [ ]:
ws_diagnostic.result.show()